In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((256,256)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    datasets.ImageFolder("../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train", data_transforms['train']),
    'validation': 
    datasets.ImageFolder("../input/asl-alphabet-new-test/asl-alphabet-test", data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True),
#                                 num_workers=0),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=16,
                                shuffle=False)
#                                 num_workers=0)  # for Kaggle
}

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 512), nn.ReLU(inplace=True),
               nn.Linear(512, 128), nn.ReLU(inplace=True),
               nn.Linear(128, 29)).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [7]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [8]:
model_trained = train_model(model, criterion, optimizer, num_epochs=4)

Epoch 1/4
----------
train loss: 1.1519, acc: 0.6289
validation loss: 5.6275, acc: 0.1701
Epoch 2/4
----------
train loss: 0.5864, acc: 0.8028
validation loss: 6.4831, acc: 0.1770
Epoch 3/4
----------
train loss: 0.4631, acc: 0.8424
validation loss: 5.9133, acc: 0.2138
Epoch 4/4
----------
train loss: 0.3986, acc: 0.8639
validation loss: 6.4332, acc: 0.2172


In [9]:
for param in model.parameters():
    param.requires_grad = True  
    
model_trained = train_model(model, criterion, optimizer, num_epochs=4)

Epoch 1/4
----------
train loss: 0.3560, acc: 0.8778
validation loss: 6.6725, acc: 0.2080
Epoch 2/4
----------
train loss: 0.3253, acc: 0.8892
validation loss: 7.5847, acc: 0.2023
Epoch 3/4
----------
train loss: 0.3037, acc: 0.8963
validation loss: 7.0820, acc: 0.2092
Epoch 4/4
----------


KeyboardInterrupt: 